In [1]:
from __future__ import print_function 
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import datasets, models, transforms

from Data.DataLoader import *
from models.Models import *

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

%matplotlib inline


# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using:",device)

PyTorch Version:  1.12.1
Torchvision Version:  0.13.1
Using: cuda:0


In [2]:
#Global variables
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)

resize_size=256
input_size = 224

    
#Transformation 1 (Basic)
data_transforms1 = {
    'train': transforms.Compose([
        transforms.Resize(resize_size), #Establim la mida a 224
        transforms.CenterCrop(input_size), #Centrem el tallat de les imatges
        transforms.ToTensor(),
        normalize
    ]),
    'val': transforms.Compose([
        transforms.Resize(resize_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        normalize
    ]),
    'test': transforms.Compose([
        transforms.Resize(resize_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        normalize
    ]),
}


#Transformation 2 (Grey Scale)
data_transforms2 = {
    'train': transforms.Compose([
        transforms.Resize(resize_size),
        transforms.CenterCrop(input_size),
        transforms.Grayscale(3), #Output channels 3, RGB but in gray scale
        transforms.ToTensor()

    ]),
    'val': transforms.Compose([
        transforms.Resize(resize_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        normalize
    ]),
    'test': transforms.Compose([
        transforms.Resize(resize_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        normalize
    ]),
}

#Transformation 3 (Complete one)
data_transforms3 = {
    'train': transforms.Compose([
        transforms.Resize(resize_size),
        transforms.CenterCrop(input_size),
        transforms.RandomVerticalFlip(p=0.5), #Girem verticalment
        transforms.RandomHorizontalFlip(p=0.5), #Girem horitzontalment
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)), #Apliquem un blur
        transforms.ToTensor(),
        normalize
    ]),
    'val': transforms.Compose([
        transforms.Resize(resize_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        normalize
    ]),
    'test': transforms.Compose([
        transforms.Resize(resize_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        normalize
    ]),
}

#Transformation 4 (jitter)
data_transforms4 = {
    'train': transforms.Compose([
        transforms.Resize(resize_size),
        transforms.CenterCrop(input_size),
        transforms.ColorJitter(brightness=.5, hue=.3), #Jitter color
        transforms.ToTensor(),
        normalize
    ]),
    'val': transforms.Compose([
        transforms.Resize(resize_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        normalize
    ]),
    'test': transforms.Compose([
        transforms.Resize(resize_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        normalize
    ]),
}

#transform 5

channel_mean = torch.Tensor([0.485, 0.456, 0.406])
channel_std = torch.Tensor([0.229, 0.224, 0.225])

data_transforms5={
    'train': transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),

    transforms.RandomHorizontalFlip(p=0.6),
    transforms.RandomRotation(degrees=(30)),

    transforms.ToTensor(),
    transforms.Normalize(mean=channel_mean, std=channel_std),
]),
    'val': transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=channel_mean, std=channel_std),
])
}

In [3]:
batch_size=12
num_workers=8
DataLoader=DogsDatasetDataloders(data_transforms4, batch_size, num_workers)

Using:  cuda:0


In [4]:
DataLoader

{'train': <torch.utils.data.dataloader.DataLoader at 0x271e4a67bb0>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x271d88c96a0>}

In [5]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    acc_history = {"train": [], "val": []}
    losses = {"train": [], "val": []}

    # we will keep a copy of the best weights so far according to validation accuracy
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    losses[phase].append(loss.item())

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            
            acc_history[phase].append(epoch_acc.item())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, acc_history, losses

In [6]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

# Regnet 16

In [12]:
# Number of classes in the dataset
num_classes = 120

# Initialize the model
model, input_size = initialize_model_regnet_x_16(num_classes)

In [13]:
# Send the model to GPU
model = model.to(device)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Number of epochs to train for 
num_epochs = 15

#optimizer_ft = optim.Adam(model.parameters(), lr=0.001)

# Train and evaluate
#model1, hist1, losses1 = train_model(model, DataLoader, criterion, optimizer_ft, num_epochs=num_epochs)

In [14]:
def get_accuracy(output, label):
    output = output.to("cpu")
    label = label.to("cpu")

    sm = F.softmax(output, dim=1)
    _, index = torch.max(sm, dim=1)
    return torch.sum((label == index)) / label.size()[0]

In [15]:
def train(model, dataloader):
    model.train()
    running_loss = 0.0
    total_loss = 0.0
    running_acc = 0.0
    total_acc = 0.0

    for batch_idx, (batch_img, batch_label) in enumerate(dataloader):

        batch_img = batch_img.to(device)
        batch_label = batch_label.to(device)

        optimizer.zero_grad()
        output = model(batch_img)
        loss = criterion(output, batch_label)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        total_loss += loss.item()

        acc = get_accuracy(output, batch_label)
        running_acc += acc
        total_acc += acc

        if batch_idx % 100 == 0 and batch_idx != 0:
            print(f"[step: {batch_idx:4d}/{len(dataloader)}] loss: {running_loss / 100:.3f}")
            running_loss = 0.0
            running_acc = 0.0
    
    return total_loss / len(dataloader), total_acc / len(dataloader)

def validate(model, dataloader):
    model.eval()
    total_loss = 0.0
    total_acc = 0.0

    for batch_idx, (batch_img, batch_label) in enumerate(dataloader):

        batch_img = batch_img.to(device)
        batch_label = batch_label.to(device)

        # optimizer.zero_grad()
        output = model(batch_img)
        loss = criterion(output, batch_label)
        # loss.backward()
        # optimizer.step()

        total_loss += loss.item()
        acc = get_accuracy(output, batch_label)
        total_acc += acc
    
    return total_loss / len(dataloader), total_acc / len(dataloader)

In [ ]:
EPOCHS = 20
train_loss_history = []
valid_loss_history = []

train_acc_history = []
valid_acc_history = []

optimizer = optim.Adam(model.parameters(), lr=0.005)

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, DataLoader["train"])
    valid_loss, valid_acc = validate(model, DataLoader["val"])
    print(f"Epoch: {epoch:2d}, training loss: {train_loss:.3f}, training acc: {train_acc:.3f} validation loss: {valid_loss:.3f}, validation acc: {valid_acc:.3f}")

    train_loss_history.append(train_loss)
    valid_loss_history.append(valid_loss)

    train_acc_history.append(train_acc)
    valid_acc_history.append(valid_acc)

    if valid_loss <= min(valid_loss_history):
        torch.save(model.state_dict(), (os.getcwd()+"/models/RegNet_X_16GfFeatureExtractionTransform4_Adam"))

[step:  100/597] loss: 7.588
[step:  200/597] loss: 4.480
[step:  300/597] loss: 3.923
[step:  400/597] loss: 4.328
[step:  500/597] loss: 3.874
Epoch:  0, training loss: 4.671, training acc: 0.345 validation loss: 2.046, validation acc: 0.655
[step:  100/597] loss: 2.965
[step:  200/597] loss: 3.074
[step:  300/597] loss: 3.584
[step:  400/597] loss: 3.705
[step:  500/597] loss: 3.358


In [ ]:
torch.save(model1.state_dict(),(os.getcwd()+"/models/ViTFeatureExtractTransform4"))

In [ ]:
# plot the losses and accuracies
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

ax1.plot(losses1["train"], label="training loss")
ax1.plot(losses1["val"], label="validation loss")
ax1.legend()

ax2.plot(hist1["train"],label="training accuracy")
ax2.plot(hist1["val"],label="val accuracy")
ax2.legend()

plt.show()   

# ConvNet HAY Q PASARLE LOS INPUUUUTS

In [ ]:
# Number of classes in the dataset
num_classes = 120

# Initialize the model
model, input_size = ConvNet(num_classes)

In [ ]:
# Send the model to GPU
model = model.to(device)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Number of epochs to train for 
num_epochs = 15

#optimizer_ft = optim.Adam(model.parameters(), lr=0.001)

# Train and evaluate
#model1, hist1, losses1 = train_model(model, DataLoader, criterion, optimizer_ft, num_epochs=num_epochs)

In [ ]:
EPOCHS = 20
train_loss_history = []
valid_loss_history = []

train_acc_history = []
valid_acc_history = []

optimizer = optim.Adam(model.parameters(), lr=0.005)

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, DataLoader["train"])
    valid_loss, valid_acc = validate(model, DataLoader["val"])
    print(f"Epoch: {epoch:2d}, training loss: {train_loss:.3f}, training acc: {train_acc:.3f} validation loss: {valid_loss:.3f}, validation acc: {valid_acc:.3f}")

    train_loss_history.append(train_loss)
    valid_loss_history.append(valid_loss)

    train_acc_history.append(train_acc)
    valid_acc_history.append(valid_acc)

    if valid_loss <= min(valid_loss_history):
        torch.save(model.state_dict(), (os.getcwd()+"/models/RegNet_X_16GfFeatureExtractionTransform4_Adam"))

In [ ]:
torch.save(model1.state_dict(),(os.getcwd()+"/models/ViTFeatureExtractTransform4"))

In [ ]:
# plot the losses and accuracies
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

ax1.plot(losses1["train"], label="training loss")
ax1.plot(losses1["val"], label="validation loss")
ax1.legend()

ax2.plot(hist1["train"],label="training accuracy")
ax2.plot(hist1["val"],label="val accuracy")
ax2.legend()

plt.show()   

# Net HAY Q PASARLE LOS INPUUUUTS

In [ ]:
# Number of classes in the dataset
num_classes = 120

# Initialize the model
model, input_size = Net(num_classes)

In [ ]:
# Send the model to GPU
model = model.to(device)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Number of epochs to train for 
num_epochs = 15

#optimizer_ft = optim.Adam(model.parameters(), lr=0.001)

# Train and evaluate
#model1, hist1, losses1 = train_model(model, DataLoader, criterion, optimizer_ft, num_epochs=num_epochs)

In [ ]:
EPOCHS = 20
train_loss_history = []
valid_loss_history = []

train_acc_history = []
valid_acc_history = []

optimizer = optim.Adam(model.parameters(), lr=0.005)

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, DataLoader["train"])
    valid_loss, valid_acc = validate(model, DataLoader["val"])
    print(f"Epoch: {epoch:2d}, training loss: {train_loss:.3f}, training acc: {train_acc:.3f} validation loss: {valid_loss:.3f}, validation acc: {valid_acc:.3f}")

    train_loss_history.append(train_loss)
    valid_loss_history.append(valid_loss)

    train_acc_history.append(train_acc)
    valid_acc_history.append(valid_acc)

    if valid_loss <= min(valid_loss_history):
        torch.save(model.state_dict(), (os.getcwd()+"/models/RegNet_X_16GfFeatureExtractionTransform4_Adam"))

In [ ]:
torch.save(model1.state_dict(),(os.getcwd()+"/models/ViTFeatureExtractTransform4"))

In [ ]:
# plot the losses and accuracies
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

ax1.plot(losses1["train"], label="training loss")
ax1.plot(losses1["val"], label="validation loss")
ax1.legend()

ax2.plot(hist1["train"],label="training accuracy")
ax2.plot(hist1["val"],label="val accuracy")
ax2.legend()

plt.show()   

# ResNet 50 

In [ ]:
# Number of classes in the dataset
num_classes = 120

# Initialize the model
res = torchvision.models.resnet50(pretrained=True)
for param in res.parameters():
    param.requires_grad=False

model_final = ResNet50(base_model=res, base_out_features=res.fc.out_features, num_classes=120)
model_final = model_final.to(device)

In [ ]:
# Send the model to GPU
model = model.to(device)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Number of epochs to train for 
num_epochs = 15

#optimizer_ft = optim.Adam(model.parameters(), lr=0.001)

# Train and evaluate
#model1, hist1, losses1 = train_model(model, DataLoader, criterion, optimizer_ft, num_epochs=num_epochs)

In [ ]:
EPOCHS = 20
train_loss_history = []
valid_loss_history = []

train_acc_history = []
valid_acc_history = []

optimizer = optim.Adam(model.parameters(), lr=0.005)

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, DataLoader["train"])
    valid_loss, valid_acc = validate(model, DataLoader["val"])
    print(f"Epoch: {epoch:2d}, training loss: {train_loss:.3f}, training acc: {train_acc:.3f} validation loss: {valid_loss:.3f}, validation acc: {valid_acc:.3f}")

    train_loss_history.append(train_loss)
    valid_loss_history.append(valid_loss)

    train_acc_history.append(train_acc)
    valid_acc_history.append(valid_acc)

    if valid_loss <= min(valid_loss_history):
        torch.save(model.state_dict(), (os.getcwd()+"/models/RegNet_X_16GfFeatureExtractionTransform4_Adam"))

In [ ]:
torch.save(model1.state_dict(),(os.getcwd()+"/models/ViTFeatureExtractTransform4"))

In [ ]:
# plot the losses and accuracies
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

ax1.plot(losses1["train"], label="training loss")
ax1.plot(losses1["val"], label="validation loss")
ax1.legend()

ax2.plot(hist1["train"],label="training accuracy")
ax2.plot(hist1["val"],label="val accuracy")
ax2.legend()

plt.show()   

# XV3V2N HAY Q PASARLE LOS INPUUUUTS

In [ ]:
# Number of classes in the dataset
num_classes = 120

# Initialize the model
model, input_size = XV3V2N(num_classes)

In [ ]:
# Send the model to GPU
model = model.to(device)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Number of epochs to train for 
num_epochs = 15

#optimizer_ft = optim.Adam(model.parameters(), lr=0.001)

# Train and evaluate
#model1, hist1, losses1 = train_model(model, DataLoader, criterion, optimizer_ft, num_epochs=num_epochs)

In [ ]:
EPOCHS = 20
train_loss_history = []
valid_loss_history = []

train_acc_history = []
valid_acc_history = []

optimizer = optim.Adam(model.parameters(), lr=0.005)

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, DataLoader["train"])
    valid_loss, valid_acc = validate(model, DataLoader["val"])
    print(f"Epoch: {epoch:2d}, training loss: {train_loss:.3f}, training acc: {train_acc:.3f} validation loss: {valid_loss:.3f}, validation acc: {valid_acc:.3f}")

    train_loss_history.append(train_loss)
    valid_loss_history.append(valid_loss)

    train_acc_history.append(train_acc)
    valid_acc_history.append(valid_acc)

    if valid_loss <= min(valid_loss_history):
        torch.save(model.state_dict(), (os.getcwd()+"/models/RegNet_X_16GfFeatureExtractionTransform4_Adam"))

In [ ]:
torch.save(model1.state_dict(),(os.getcwd()+"/models/ViTFeatureExtractTransform4"))

In [ ]:
# plot the losses and accuracies
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

ax1.plot(losses1["train"], label="training loss")
ax1.plot(losses1["val"], label="validation loss")
ax1.legend()

ax2.plot(hist1["train"],label="training accuracy")
ax2.plot(hist1["val"],label="val accuracy")
ax2.legend()

plt.show()   

# ConcatIXIRN HAY Q PASARLE LOS INPUUUUTS

In [ ]:
# Number of classes in the dataset
num_classes = 120

# Initialize the model
model, input_size = ConcatIXIRN(num_classes)

In [ ]:
# Send the model to GPU
model = model.to(device)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Number of epochs to train for 
num_epochs = 15

#optimizer_ft = optim.Adam(model.parameters(), lr=0.001)

# Train and evaluate
#model1, hist1, losses1 = train_model(model, DataLoader, criterion, optimizer_ft, num_epochs=num_epochs)

In [ ]:
EPOCHS = 20
train_loss_history = []
valid_loss_history = []

train_acc_history = []
valid_acc_history = []

optimizer = optim.Adam(model.parameters(), lr=0.005)

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, DataLoader["train"])
    valid_loss, valid_acc = validate(model, DataLoader["val"])
    print(f"Epoch: {epoch:2d}, training loss: {train_loss:.3f}, training acc: {train_acc:.3f} validation loss: {valid_loss:.3f}, validation acc: {valid_acc:.3f}")

    train_loss_history.append(train_loss)
    valid_loss_history.append(valid_loss)

    train_acc_history.append(train_acc)
    valid_acc_history.append(valid_acc)

    if valid_loss <= min(valid_loss_history):
        torch.save(model.state_dict(), (os.getcwd()+"/models/RegNet_X_16GfFeatureExtractionTransform4_Adam"))

In [ ]:
torch.save(model1.state_dict(),(os.getcwd()+"/models/ViTFeatureExtractTransform4"))

In [ ]:
# plot the losses and accuracies
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

ax1.plot(losses1["train"], label="training loss")
ax1.plot(losses1["val"], label="validation loss")
ax1.legend()

ax2.plot(hist1["train"],label="training accuracy")
ax2.plot(hist1["val"],label="val accuracy")
ax2.legend()

plt.show()   